#Install Dependencies

In [1]:
%pip install fastapi uvicorn pyngrok nest-asyncio

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 1.7 MB/s eta 0:00:01
   -------------------- ------------------- 1.0/2.0 MB 1.7 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.0 MB 1.7 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 1.7 MB/s eta 0:00:01
   ------------------------------------ --- 1.8/2.0 MB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 1.5 MB/s  0:00:01
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)

   -------- -------------------------------  3/15 [


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


#Setup FastAPI/Ngrok

In [2]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Allow FastAPI to run inside Colab's event loop
nest_asyncio.apply()

# Set up ngrok with your token
ngrok.set_auth_token("34AQXQUgqFjkJH9xEuP5y5xeHm2_75EaDxY3oEz1EUTgGc63S")

# Create FastAPI app
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # for testing
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/hello")
def hello():
    return {"message": "Hello from FastAPI in Colab!"}

# Create tunnel
ngrok.kill()  #makes sure multiple connections arent made, i aint paying money to have more network connections
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run uvicorn manually inside existing loop
config = uvicorn.Config(app=app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()




INFO:     Started server process [11592]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://dexter-hydrarch-karla.ngrok-free.dev" -> "http://localhost:8000"
INFO:     45.32.56.47:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     45.32.56.47:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [11592]


#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#Tenby10 Backend Experiment
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [3]:
%pip install cerebras-cloud-sdk pandas gradio

  Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-win_amd64.whl.metadata (6.9 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/63.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/63.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/63.5 MB 1.6 MB/s eta 0:00:39
   ---------------------------------------- 0.5/63.5 MB 1.6 MB/s eta 0:00:39
   ---------------------------------------- 0.8/63.5 MB 1.0 MB/s eta 0:01:02
    --------------------------------------- 1.0/63.5 MB 1.0 MB/s eta 0:01:00
    --------------------------------------- 1.3/63.5 MB 1.1 MB/s eta 0:00:56
    --------------------------------------- 1.6/63.5 MB 

  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from cerebras.cloud.sdk import Cerebras
import pandas as pd
import json
import gradio as gr


c:\Users\Yuna\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
client = Cerebras(api_key=os.environ["CEREBRAS_API_KEY_JOSEPH"])

def generate_flashcards(vocab_list, target_language="Spanish"):
    results = []

    for word in vocab_list:
        prompt = f"""
        You are a multilingual language learning assistant.
        For the term "{word}", translate it into {target_language}, and give one natural example sentence in that language.
        Respond ONLY in valid JSON with this exact structure:
        {{
            "term": "{word}",
            "translation": "...",
            "example_sentence": "..."
        }}
        """

        try:
            completion = client.chat.completions.create(
                model="qwen-3-235b-a22b-instruct-2507",  # or another available model
                messages=[
                    {"role": "system", "content": "You are a helpful multilingual assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_completion_tokens=500,
                temperature=0.7,
                top_p=0.9,
                stream=False
            )

            # The returned object has the message content in a slightly different place than OpenAI
            content = completion.choices[0].message.content.strip()
            try:
                data = json.loads(content)
            except:
                # If the model returns text instead of JSON, fallback parser
                data = {
                    "term": word,
                    "translation": content.split("\n")[0] if "\n" in content else content,
                    "example_sentence": ""
                }

            results.append({
                "Grammar/Vocab": data.get("term", word),
                "Meaning": data.get("translation", ""),
                "Example Sentence": data.get("example_sentence", "")
            })

        except Exception as e:
            results.append({
                "Grammar/Vocab": word,
                "Meaning": f"Error: {e}",
                "Example Sentence": ""
            })

    return pd.DataFrame(results)

In [8]:
def run_flashcard_generator(text, language):
    vocab_list = [w.strip() for w in text.split("\n") if w.strip()]
    df = generate_flashcards(vocab_list, language)
    csv_path = "anki_flashcards.csv"
    df.to_csv(csv_path, index=False)
    return df, csv_path

demo = gr.Interface(
    fn=run_flashcard_generator,
    inputs=[
        gr.Textbox(label="Enter vocab terms (one per line)", lines=6, placeholder="e.g.\nla organización\nel animal\nla tecnología"),
        gr.Radio(["Spanish", "French", "Japanese"], label="Target Language")
    ],
    outputs=[
        gr.Dataframe(label="Generated Flashcards"),
        gr.File(label="Download CSV")
    ],
    title="TEN by10 – AI Flashcard Generator",
    description="Generate Anki flashcards with translations and example sentences."
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
